# Parallelism Strategy

In [ ]:
#| default_exp strategy

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
import einops

In [ ]:
#| export
class BaseStrategy:
    """Base class for mapping strategy."""
    
    def __call__(
        self, 
        fn: Callable, # Function to generate cf for a single input
        xs: Array, # Input instances to be explained
        pred_fn: Callable[[Array], Array],
        y_targets: Array,
        rng_keys: Iterable[jrand.PRNGKey],
        **kwargs
    ) -> Array: # Generated counterfactual explanations
        raise NotImplementedError
    
    __ALL__ = ["__call__"]


In [ ]:
#| export
class IterativeStrategy(BaseStrategy):
    """Iterativly generate counterfactuals."""

    def __call__(
        self, 
        fn: Callable, # Function to generate cf for a single input
        xs: Array, # Input instances to be explained
        pred_fn: Callable[[Array], Array],
        y_targets: Array,
        rng_keys: Iterable[jrand.PRNGKey],
        **kwargs
    ) -> Array: # Generated counterfactual explanations
        
        assert xs.ndim == 2
        cfs = jnp.stack([fn(xs[i], pred_fn=pred_fn, y_target=y_targets[i], rng_key=rng_keys[i], **kwargs) 
            for i in range(xs.shape[0])])
        return cfs


In [ ]:
#| export
class VmapStrategy(BaseStrategy):
    """Generate counterfactuals via `jax.vmap`."""

    def __call__(
        self, 
        fn: Callable, # Function to generate cf for a single input
        xs: Array, # Input instances to be explained
        pred_fn: Callable[[Array], Array],
        y_targets: Array,
        rng_keys: Iterable[jrand.PRNGKey],
        **kwargs
    ) -> Array: # Generated counterfactual explanations
        
        def partial_fn(x, y_target, rng_key):
            return fn(x, pred_fn=pred_fn, y_target=y_target, rng_key=rng_key, **kwargs)
        
        assert xs.ndim == 2
        cfs = jax.vmap(partial_fn)(xs, y_targets, rng_keys)
        return cfs


In [ ]:
#| exporti
def _pad_divisible_X(
    xs: Array,
    n_devices: int
):
    """Pad `X` to be divisible by `n_devices`."""
    if xs.shape[0] % n_devices != 0:
        pad_size = n_devices - xs.shape[0] % n_devices
        xs_pad = einops.repeat(
            xs[-1:], "n ... -> (pad n) ...", pad=pad_size
        )
        xs = jnp.concatenate([xs, xs_pad])
    X_padded = xs.reshape(n_devices, -1, *xs.shape[1:])
    return X_padded


In [ ]:
#| hide
xs = jnp.ones((5, 29))
X_padded = _pad_divisible_X(xs, 2)
assert X_padded.shape == (2, 3, 29)
assert xs.sum() < X_padded.sum()

xs = jax.nn.one_hot(jnp.arange(5), 29)
X_padded = _pad_divisible_X(xs, 2)
assert xs.shape == (5, 29)
assert X_padded.shape == (2, 3, 29)
assert np.array_equal(
    X_padded.sum(-1), np.ones((2, 3))
)

xs = jrand.split(jrand.PRNGKey(0), 5)
X_padded = _pad_divisible_X(xs, 2)
assert X_padded.shape == (2, 3, 2)

xs = jnp.ones((5, 29))
X_padded = _pad_divisible_X(xs, 6)
assert X_padded.shape == (6, 1, 29)

xs = jnp.ones((5, 29))
X_padded = _pad_divisible_X(xs, 1)
assert X_padded.shape == (1, 5, 29)



In [ ]:
#| export
class PmapStrategy(BaseStrategy):
    def __init__(
        self, 
        n_devices: int = None, # Number of devices. If None, use all available devices
        strategy: str = 'auto', # Strategy to generate counterfactuals
        **kwargs
    ):
        self.strategy = strategy
        self.n_devices = n_devices or jax.device_count()

    def __call__(
        self, 
        fn: Callable, # Function to generate cf for a single input
        xs: Array, # Input instances to be explained
        pred_fn: Callable[[Array], Array],
        y_targets: Array,
        rng_keys: Iterable[jrand.PRNGKey],
        **kwargs
    ) -> Array: # Generated counterfactual explanations
        
        def partial_fn(x, y_target, rng_key, **kwargs):
            return fn(x, pred_fn=pred_fn, y_target=y_target, rng_key=rng_key, **kwargs)

        assert xs.ndim == 2
        X_padded = _pad_divisible_X(xs, self.n_devices)
        y_targets = _pad_divisible_X(y_targets, self.n_devices)
        rng_keys = _pad_divisible_X(rng_keys, self.n_devices)
        cfs = jax.pmap(jax.vmap(partial_fn))(X_padded, y_targets, rng_keys)
        cfs = cfs.reshape(-1, *cfs.shape[2:])
        cfs = cfs[:xs.shape[0]]
        return cfs


In [ ]:
#| exporti
def _pad_xs(
    xs: Array, pad_size: int, batch_size: int
):
    """Pad `X` to be divisible by `n_devices`."""
    xs_pad = einops.repeat(
        xs[-1:], "n ... -> (pad n) ...", pad=pad_size
    )
    xs = jnp.concatenate([xs, xs_pad])
    xs = einops.rearrange(xs, "(b n) ... -> b n ...", b=batch_size)
    return xs

def _batched_generation(
    gs_fn: Callable, # Generation strategy function
    cf_fn: Callable, # Function to generate cf for a single input
    xs: Array, # Input instances to be explained
    pred_fn: Callable[[Array], Array],
    y_targets: Array,
    rng_keys: Iterable[jrand.PRNGKey],
    batch_size: int,
    **kwargs
) -> Array: # Generated counterfactual explanations
    """Batched  of counterfactuals."""

    def gs_fn_partial(state):
        x, y_target, rng_key = state
        return gs_fn(cf_fn, x, pred_fn, y_target, rng_key, **kwargs)
    
    assert xs.ndim == 2, f"X must be a 2D array, got {xs.ndim}D array"
    x_shape = xs.shape
    batch_size = min(batch_size, x_shape[0])
    # pad X to be divisible by batch_size
    pad_size = batch_size - (xs.shape[0] % batch_size)
    xs = _pad_xs(xs, pad_size, batch_size)
    y_targets = _pad_xs(y_targets, pad_size, batch_size)
    rng_keys = _pad_xs(rng_keys, pad_size, batch_size)
    # generate cfs via lax.map
    cfs = lax.map(gs_fn_partial, (xs, y_targets, rng_keys))
    # cfs = cfs.reshape(-1, *x_shape[1:])[:x_shape[0]]
    cfs = einops.rearrange(cfs, "n b ... k -> (n b) ... k")
    cfs = cfs[:x_shape[0]]
    return cfs

In [ ]:
#| export
class BatchedVmapStrategy(BaseStrategy):
    """Auto-batching for generate counterfactuals via `jax.vmap`."""
    def __init__(self, batch_size: int):
        self.batch_size = batch_size

    def __call__(
        self, 
        fn: Callable, # Function to generate cf for a single input
        xs: Array, # Input instances to be explained
        pred_fn: Callable[[Array], Array],
        y_targets: Array,
        rng_keys: Iterable[jrand.PRNGKey],
        **kwargs
    ) -> Array: # Generated counterfactual explanations
        vmap_g = VmapStrategy()    
        cfs = _batched_generation(
            vmap_g, fn, xs, pred_fn, y_targets, rng_keys, self.batch_size, **kwargs
        )
        return cfs


In [ ]:
#| export
class BatchedPmapStrategy(BaseStrategy):
    """Auto-batching for generate counterfactuals via `jax.vmap`."""
    def __init__(self, batch_size: int, n_devices: int = None):
        self.batch_size = batch_size
        self.n_devices = n_devices

    def __call__(
        self, 
        fn: Callable, # Function to generate cf for a single input
        xs: Array, # Input instances to be explained
        pred_fn: Callable[[Array], Array],
        y_targets: Array,
        rng_keys: Iterable[jrand.PRNGKey],
        **kwargs
    ) -> Array: # Generated counterfactual explanations
        pmap_g = PmapStrategy(self.n_devices)
        cfs = _batched_generation(
            pmap_g, fn, xs, pred_fn, y_targets, rng_keys, self.batch_size, **kwargs
        )
        return cfs


In [ ]:
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'

w = jrand.normal(jrand.PRNGKey(0), (100, 100))
xs = jrand.normal(jrand.PRNGKey(0), (1000, 100))

@jit
def pred_fn(x): return jnp.dot(x, w.T)

def f(x, pred_fn=None, y_target=None, rng_key=None, **kwargs):
    return pred_fn(x) + jrand.normal(rng_key, (1,))

rng_keys = jrand.split(jrand.PRNGKey(0), 1000)
y_targets = jnp.ones((1000, 100))

iter_gen = IterativeStrategy()
vmap_gen = VmapStrategy()
pmap_gen = PmapStrategy()
bvmap_gen = BatchedVmapStrategy(128)
bpmap_gen = BatchedPmapStrategy(128)


In [ ]:
cf_iter = iter_gen(f, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)

In [ ]:
cf_vmap = vmap_gen(f, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)

In [ ]:
cf_pmap = pmap_gen(f, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)


In [ ]:
cf_bvmap = bvmap_gen(f, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)


In [ ]:
#| hide
# check when batch_size > X.shape[0]
_bvmap_gen = BatchedVmapStrategy(1280)
_cf_bvmap = _bvmap_gen(f, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)
assert jnp.allclose(cf_bvmap, _cf_bvmap, atol=1e-4)


In [ ]:
#| hide
cf_bpmap = bpmap_gen(f, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)

In [ ]:
#| hide
assert jnp.allclose(cf_iter, cf_vmap, atol=1e-4)
assert jnp.allclose(cf_iter, cf_bvmap, atol=1e-4)
assert jnp.allclose(cf_iter, cf_pmap, atol=1e-4)
assert jnp.allclose(cf_iter, cf_bpmap, atol=1e-4)

In [ ]:
def f_mul(x, pred_fn=None, **kwargs):
    cf = pred_fn(x)
    return einops.repeat(cf, 'k -> c k', c=5)

In [ ]:
cf_iter = iter_gen(f_mul, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)
cf_vmap = vmap_gen(f_mul, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)
cf_pmap = pmap_gen(f_mul, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)
cf_bvmap = bvmap_gen(f_mul, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)
cf_bpmap = bpmap_gen(f_mul, xs, pred_fn=pred_fn, y_targets=y_targets, rng_keys=rng_keys)

assert jnp.allclose(cf_iter, cf_vmap, atol=1e-4)
assert jnp.allclose(cf_iter, cf_bvmap, atol=1e-4)
assert jnp.allclose(cf_iter, cf_pmap, atol=1e-4)
assert jnp.allclose(cf_iter, cf_bpmap, atol=1e-4)
assert cf_bvmap.shape == (xs.shape[0], 5, xs.shape[1])

In [ ]:
#| export
class StrategyFactory(object):
    """Factory class for Parallelism Strategy."""

    __strategy_map = {
        'iter': IterativeStrategy(),
        'vmap': VmapStrategy(),
        'pmap': PmapStrategy(),
    }

    def __init__(self) -> None:
        raise ValueError("This class should not be instantiated.")
        
    @staticmethod
    def get_default_strategy() -> BaseStrategy:
        """Get default strategy."""
        return VmapStrategy()

    @classmethod
    def get_strategy(cls, strategy: str | BaseStrategy) -> BaseStrategy:
        """Get strategy."""
        if isinstance(strategy, BaseStrategy):
            return strategy
        elif isinstance(strategy, str) and strategy in cls.__strategy_map:
            return cls.__strategy_map[strategy]
        else:
            raise ValueError(f"Invalid strategy: {strategy}")
        
    __ALL__ = ["get_default_strategy", "get_strategy"]

In [ ]:
it = StrategyFactory.get_strategy('iter')
vm = StrategyFactory.get_strategy('vmap')
pm = StrategyFactory.get_strategy('pmap')
default = StrategyFactory.get_default_strategy()
cus = StrategyFactory.get_strategy(VmapStrategy())

assert isinstance(it, IterativeStrategy)
assert isinstance(vm, VmapStrategy)
assert isinstance(pm, PmapStrategy)
assert isinstance(default, VmapStrategy)
assert isinstance(cus, VmapStrategy)